In [1]:
import os
import json
import math
import sys
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

from accelerate import Accelerator

from config import Config
from audiomodel_seperation import AudioProcessing
from audiodataset_seperation import SeperationDataset

def make_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def build_model(cfg):
        from audiocraft.models.loaders import load_compression_model, load_lm_model
        """Instantiate models and optimizer."""     
        compression_model = load_compression_model('facebook/audiogen-medium', device=cfg.device)
        lm = load_lm_model('facebook/audiogen-medium', device=cfg.device)
        return compression_model, lm

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cu118)
    Python  3.10.13 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [2]:
cfg = Config()

model_path = "./sep_models/model_epoch_0_12.pth"

base_path = "./csv_files/"
train_data_path = f"{base_path}/train_dataset_epidemic_sub.csv"
eval_data_path = f"{base_path}/eval_dataset_epidemic_sub.csv"

cfg.update(train_data_path=train_data_path, eval_data_path=eval_data_path, batch_size=4)


save_path = "./test"

make_dir(save_path)

cfg.update(**{"save_path": save_path})


eval_dataset = SeperationDataset(cfg, train=False)
eval_dataloader = DataLoader(eval_dataset, batch_size=1, shuffle=False, num_workers=8)

# compression_model, lm = build_model(cfg)
# model = AudioProcessing(cfg, lm)

# model.load_state_dict(torch.load(model_path))

# torch.cuda.empty_cache()

In [5]:
s, p, g, l, w2 = eval_dataset[4]

print(p)

Remove 'A whoosh sound effect.'


In [6]:
from IPython.display import Audio

display(Audio(data=s[0].numpy(), rate=cfg.sample_rate))
display(Audio(data=g[0].numpy(), rate=cfg.sample_rate))
display(Audio(data=w2[0].numpy(), rate=cfg.sample_rate))

In [8]:
cfg.update(duration=3, batch_size=1)

In [9]:
model.eval()
compression_model.eval()
audio_num = 0
for test_step, (synthesized_wav, prompts, ground_truth, lengths) in enumerate(eval_dataloader):
    if audio_num>=5:
        break
    with torch.no_grad():
        # synthesized_wav와 prompts가 사용되어야 한다.
        prompt = prompts[0]
        print(prompt)
        
        audio_tokens, _ = compression_model.encode(synthesized_wav.to("cuda"))
        print("audio_tokens : ", audio_tokens.shape)
        
        gen_tokens, gen_audio = model.inference(audio_tokens, prompts, compression_model)
        audio_filename = f"{prompt}_{audio_num}.wav"
        model.save_audio(gen_audio, audio_filename, cfg)
        
        from IPython.display import Audio
        # 합쳐진거, GT, 생성된 소리 다 비교
        display(Audio(data=synthesized_wav[0].numpy(), rate=cfg.sample_rate))
        display(Audio(data=ground_truth[0].numpy(), rate=cfg.sample_rate))
        display(Audio(data=gen_audio[0].detach().cpu().numpy(), rate=cfg.sample_rate))

        audio_num += 1

        del gen_tokens
        del gen_audio
        del synthesized_wav
        del audio_tokens
        
        torch.cuda.empty_cache()

Remove 'A short snare drum sound.'
audio_tokens :  torch.Size([1, 4, 150])
initial_tokens :  torch.Size([1, 4, 150])


AssertionError: 

In [25]:
del audio_tokens
del gen_tokens
del gen_audio

torch.cuda.empty_cache()